# Extraindo os dados

In [1]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv"
dados = pd.read_csv(uri)
dados.head()

,Unnamed: 0,mileage_per_year,model_year,price,sold
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


# Tratamento dos dados

In [2]:
a_renomear = {
    'mileage_per_year' : 'milhas_por_ano',
    'model_year' : 'ano_do_modelo',
    'price' : 'preco',
    'sold' : 'vendido'
}
dados = dados.rename(columns=a_renomear)
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


In [3]:
a_trocar = {
    'no' : 0,
    'yes' : 1
}
dados.vendido = dados.vendido.map(a_trocar)
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,1
1,1,7843,1998,40557.96,1
2,2,7109,2006,89627.50,0
3,3,26823,2015,95276.14,0
4,4,7935,2014,117384.68,1


In [4]:
from datetime import datetime

ano_atual = datetime.today().year
dados['idade_do_modelo'] = ano_atual - dados.ano_do_modelo
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo
0,0,21801,2000,30941.02,1,24
1,1,7843,1998,40557.96,1,26
2,2,7109,2006,89627.50,0,18
3,3,26823,2015,95276.14,0,9
4,4,7935,2014,117384.68,1,10


In [5]:
dados['km_por_ano'] = dados.milhas_por_ano * 1.60934
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo,km_por_ano
0,0,21801,2000,30941.02,1,24,35085.22134
1,1,7843,1998,40557.96,1,26,12622.05362
2,2,7109,2006,89627.50,0,18,11440.79806
3,3,26823,2015,95276.14,0,9,43167.32682
4,4,7935,2014,117384.68,1,10,12770.11290


In [6]:
dados = dados.drop(columns = ["Unnamed: 0", "milhas_por_ano","ano_do_modelo"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,24,35085.22134
1,40557.96,1,26,12622.05362
2,89627.50,0,18,11440.79806
3,95276.14,0,9,43167.32682
4,117384.68,1,10,12770.11290


# Validação Cruzada

## Decision Tree Classifier

In [15]:
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier

x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

SEED = 5
np.random.seed(SEED)

decision_tree = DecisionTreeClassifier(max_depth=2)
results = cross_validate(decision_tree, x, y, cv=5, return_train_score=False)
mean = results['test_score'].mean()
standard_deviation = results['test_score'].std()
print("Accuracy with Decision Tree: [%.2f%%, %.2f%%]" % ((mean - 2 * standard_deviation)*100, (mean + 2 * standard_deviation)*100))

Accuracy with Decision Tree: [75.21%, 76.35%]


## Incluindo aleatoriedade na validação cruzada

In [16]:
def print_results(results):
    mean = results['test_score'].mean()
    standard_deviation = results['test_score'].std()
    print("Accuracy mean: %.2f%%" % (mean*100))
    print("Accuracy interval: [%.2f%%, %.2f%%]" % ((mean - 2 * standard_deviation)*100, (mean + 2 * standard_deviation)*100))

In [19]:
from sklearn.model_selection import KFold

SEED = 5
np.random.seed(SEED)

cv = KFold(n_splits = 5, shuffle=True)
decision_tree = DecisionTreeClassifier(max_depth=2)
results = cross_validate(decision_tree, x, y, cv=cv, return_train_score=False)
print_results(results)

Accuracy mean: 75.78%
Accuracy interval: [74.32%, 77.24%]


## StratifiedKFold para quando as classes são desbalanceadas

A validação cruzada estratificada em K folds é uma variação da validação cruzada em K folds que garante que cada fold mantenha a mesma proporção de observações para cada classe alvo que o conjunto de dados completo. Isso é especialmente crucial para conjuntos de dados onde uma classe pode estar fortemente sub-representada.

In [20]:
from sklearn.model_selection import StratifiedKFold

SEED = 5
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 5, shuffle=True)
decision_tree = DecisionTreeClassifier(max_depth=2)
results = cross_validate(decision_tree, x, y, cv=cv, return_train_score=False)
print_results(results)

Accuracy mean: 75.76%
Accuracy interval: [74.72%, 76.80%]
